# MLP Training Module

In [1]:
import os
os.chdir('genre_classification_289a/src')

In [2]:
import torch
import torch.nn as nn
import os
from model import STN, MLP
import torch.optim as optim
import datetime

In [3]:
import numpy as np
np.random.seed(0)

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [5]:
from features import get_data_loaders, FramedFeatureDataset, FeatureDataset, DatasetSettings

### MLP Training

In [6]:
#MLP target always genre
agfs = [] #'subgenre', 'mfcc'
genre = True #False if not genre STN
    
#dataset
dataset_name = 'fma_medium'

In [7]:
settings = DatasetSettings(dataset_name, 'fma_metadata')
dataset = FramedFeatureDataset(settings,  agfs=agfs, genre=genre)
print("Num genres: ", settings.num_genres)
print(settings.genre_counts)

Num genres:  16
Rock                   6911
Electronic             6110
Experimental           2207
Hip-Hop                2109
Folk                   1477
Instrumental           1280
Pop                    1129
International          1004
Classical               598
Old-Time / Historic     510
Jazz                    380
Country                 178
Soul-RnB                154
Spoken                  118
Blues                    72
Easy Listening           21
Name: genre_top, dtype: int64


In [8]:
def get_stn_path(dataset, target):
    return '../models/DCNN_{}_{}'.format(dataset, target)

# load STNs
# @NOTE: order them by ['subgenres', 'mfcc', 'genre'] in order for analysis plots to automatically work
targets = ['subgenres', 'mfcc', 'genre']
stns = [torch.load(get_stn_path(dataset_name, target)).to(device) for target in targets]
stn_layer_dims = [None, 16, 32, 64, 64, 128, 256, 256]

#which layer to extract features from
layers = [6,6,7]
layers_str = "".join([str(l) for l in layers])

# setup MLP on GPU
mlp_input_size = np.array([stn_layer_dims[layer] for layer in layers]).sum()
mlp_output_size = settings.num_genres
mlp = MLP(mlp_input_size, mlp_output_size)
mlp.to(device)
mlp = nn.DataParallel(mlp)

## Training Parameters
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mlp.parameters(), lr=0.001)
epochs = 5
batch_size = 64
valid_split = 0.2

trainloader, validloader = get_data_loaders(dataset, batch_size, valid_split)

In [9]:
import torch.nn.functional as F
from sklearn.metrics import f1_score

def get_val_loss():
    with torch.no_grad():
        for stn in stns:
            stn.eval()
        mlp.eval()
        
        # start at a random batch and only eval that one batch
        init_idx = np.random.randint(len(validloader))

        val_loss = 0
        for j, data in enumerate(validloader, init_idx):
            inputs, labels = data[0].to(device), data[1]['genre'].to(device)

            out_intermediate = [stn.module.forward_intermediate(inputs, layers[i]) for i,stn in enumerate(stns)]
            input_mlp = torch.cat(out_intermediate, dim=1)

            out = mlp(input_mlp)
            val_loss = F.cross_entropy(out, labels)
            break
        
        return val_loss
            

# @param fast Compute validation F1 score for only one batch
def validate(fast = False):
    with torch.no_grad():
        for stn in stns:
            stn.eval()
        mlp.eval()
                
        all_pred = []
        all_true = []
        
        if fast:
            # start at a random batch if fast == True
            init_idx = np.random.randint(len(validloader))
        else:
            init_idx = 0
        
        for i, data in enumerate(validloader, init_idx):
            inputs, labels = data[0].to(device), data[1]['genre'].to(device)
            
            out_intermediate = [stn.module.forward_intermediate(inputs, layers[i]) for i,stn in enumerate(stns)]
            input_mlp = torch.cat(out_intermediate, dim=1)
            
            out = mlp(input_mlp)
            loss = F.cross_entropy(out, labels)
            
            all_pred.append(out.argmax(dim=1))
            all_true.append(labels)
            
            # only compute val accuracy for one random batch if fast == True
            if fast:
                break
            
        all_pred = torch.cat(all_pred)
        all_true = torch.cat(all_true)
        
        curr_f1 = f1_score(all_true.cpu(), all_pred.cpu(), average='micro')
        return curr_f1

In [10]:
#Train it
%time
losses = []
val_losses = []
for stn in stns:
    stn.eval()

print("Starting to train at: ", datetime.datetime.now(), " (time is +7 w.r.t. our timezone)")

#f.write('Initial Validation F1: %.6f' % validate())

mlp.train()

for epoch in range(epochs):  # loop over the dataset multiple times
    
    print('Starting epoch %d' % (epoch+1))
    
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data[0].to(device), data[1]['genre'].to(device)  #data[1]['{argument for agf being trained}']
        
        input_mlp = None
        with torch.no_grad():
            out_intermediates = [stn.module.forward_intermediate(inputs, layers[i]) for i,stn in enumerate(stns)]
            input_mlp = torch.cat(out_intermediates, dim=1)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = mlp(input_mlp)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 30 == 29:    # print every 30 mini-batches
            # @TODO average val loss over the 30 batches too?
            val_loss = get_val_loss()
            avg_loss = running_loss / 30
            print('[%d, %5d] loss: %.3f, val-loss: %.3f @ %s' % (epoch + 1,i + 1, avg_loss, val_loss, datetime.datetime.now()))
            losses.append(avg_loss)
            val_losses.append(float(val_loss))
            running_loss = 0.0

print('Finished Training')

final_f1 = validate()
np.array(val_losses).tofile(f'logs/val_losses_MLP_{dataset_name}_stn_{"_".join(targets)}_layer_{layers_str}')
np.array(losses).tofile(f'logs/losses_MLP_{dataset_name}_stn_{"_".join(targets)}_layer_{layers_str}')
np.array(final_f1).tofile(f'logs/final_MLP_{dataset_name}_stn_{"_".join(targets)}_layer_{layers_str}')

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 7.63 µs
Starting to train at:  2020-05-09 02:02:58.824310  (time is +7 w.r.t. our timezone)
Starting epoch 1
[1,    30] loss: 1.601, val-loss: 1.011 @ 2020-05-09 02:03:02.980136
[1,    60] loss: 0.853, val-loss: 0.652 @ 2020-05-09 02:03:05.917239
[1,    90] loss: 0.619, val-loss: 0.455 @ 2020-05-09 02:03:08.871687
[1,   120] loss: 0.534, val-loss: 0.364 @ 2020-05-09 02:03:11.807746
[1,   150] loss: 0.449, val-loss: 0.364 @ 2020-05-09 02:03:29.114115
[1,   180] loss: 0.362, val-loss: 0.290 @ 2020-05-09 02:03:56.647104
[1,   210] loss: 0.303, val-loss: 0.238 @ 2020-05-09 02:04:21.103603
[1,   240] loss: 0.347, val-loss: 0.197 @ 2020-05-09 02:04:43.741988
[1,   270] loss: 0.301, val-loss: 0.135 @ 2020-05-09 02:05:03.768120
[1,   300] loss: 0.314, val-loss: 0.173 @ 2020-05-09 02:05:25.265068
[1,   330] loss: 0.312, val-loss: 0.290 @ 2020-05-09 02:05:47.551788
[1,   360] loss: 0.268, val-loss: 0.106 @ 2020-05-09 02:06:05.720385
[1,   3

OSError: [Errno 12] Cannot allocate memory

### Save Model

In [ ]:
model_file = f'../models/MLP_{dataset_name}_stn_{"_".join(targets)}_layer_{layers_str}'
torch.save(mlp, model_file)

## Load & Eval Model

In [ ]:
model_file = f'../models/MLP_{dataset_name}_stn_{"_".join(targets)}_layer_{layers_str}'
mlp = torch.load(model_file)

Load losses and final accuracy:

In [ ]:
losses = np.fromfile(f'logs/losses_MLP_{dataset_name}_stn_{"_".join(targets)}_layer_{layers_str}')
val_losses = np.fromfile(f'logs/val_losses_MLP_{dataset_name}_stn_{"_".join(targets)}_layer_{layers_str}')
final_f1 = np.fromfile(f'logs/final_MLP_{dataset_name}_stn_{"_".join(targets)}_layer_{layers_str}')

In [ ]:
val_losses

## MLP Performance History
* SGM layer 4: 0.67209446
* 